In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as  scr
import anndata
import scanpy.external as sce
from matplotlib import pyplot as plt
%matplotlib inline

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')

In [ ]:
COPD = sc.read("processed_data_py/COPD_newdata_0808.h5ad")
endo = COPD[COPD.obs["cellclass"] == "Endothelial"].copy()

In [ ]:
sc.pp.highly_variable_genes(endo, n_top_genes=2000, flavor = "seurat")
endo.raw = endo
endo = endo[:, endo.var.highly_variable]
sc.pp.regress_out(endo, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(endo)
sc.tl.pca(endo, n_comps = 50)
sc.external.pp.bbknn(endo, batch_key = "sample")
sc.tl.umap(endo)
sc.pp.neighbors(endo, n_neighbors=30, n_pcs=50)
sc.tl.leiden(endo,resolution = 0.8)
sc.pl.umap(endo, color=["leiden"],size=1, legend_loc='on data')

In [ ]:
sc.pl.dotplot(endo,{"Epithelial":["EPCAM","AGER","SFTPC","SCGB3A2","FOXJ1"],
                        "endoenchymal":["COL1A1","PDGFRA","PDGFRB"],
                        "Endothleial":["PECAM1","CDH5","LYVE1","TFF3","CA4","FCN3","VWF","EDNRB","HPGD",
                                       "IGFBP3","CPE","HDAC9","COL15A1","SPRY1","PLVAP","TPD52L1"],
                        "Immune":["PTPRC","NKG7","CSF3R","FABP4"],
                       "MKI67":["TOP2A"]}, 
                  groupby="leiden",dendrogram=False,standard_scale="var",cmap='Spectral_r')

In [ ]:
endo.obs["celltype"] = "gCap"
endo.obs.loc[endo.obs["leiden"].isin(["9","10","13","15","16","19"]),"celltype"] = "Doublets"
endo.obs.loc[endo.obs["leiden"].isin(["3"]),"celltype"] = "Lymphatic"
endo.obs.loc[endo.obs["leiden"].isin(["0"]),"celltype"] = "Aerocyte"
endo.obs.loc[endo.obs["leiden"].isin(["6"]),"celltype"] = "Arterial"
endo.obs.loc[endo.obs["leiden"].isin(["5","12"]),"celltype"] = "Venous"
sc.pl.umap(endo, color=["celltype"],size=1, legend_loc='on data')

In [ ]:
sc.pl.dotplot(endo,{"Epithelial":["EPCAM","AGER","SFTPC","SCGB3A2","FOXJ1"],
                        "endoenchymal":["COL1A1","PDGFRA","PDGFRB"],
                        "Endothleial":["PECAM1","CDH5","LYVE1","TFF3","CA4","FCN3","VWF","EDNRB","HPGD","IGFBP3","CPE","HDAC9","COL15A1"],
                        "Immune":["PTPRC","NKG7","CSF3R"]}, 
                  groupby="celltype",dendrogram=False,standard_scale="var",cmap='Spectral_r')

In [ ]:
endo_meta = endo.obs[["celltype"]]
endo_meta.to_csv("tables/endo_meta.csv")
endo = endo.raw.to_adata()

In [ ]:
endo = endo[endo.obs["celltype"] != "Doublets"].copy()
sc.tl.pca(endo, n_comps = 50)
sc.external.pp.bbknn(endo, batch_key = "sample")
sc.tl.umap(endo)

In [ ]:
sc.pl.umap(endo, color=["celltype"],size=3, legend_loc='on data')

In [ ]:
sc.pl.dotplot(endo,{"Lymphatic": ["LYVE1","TFF3","CCL21"], "gCap": ["CA4","FCN3","IL7R","SLC6A4"],
                   "Aerocyte": ["CA4","EDNRB","HPGD","SOSTDC1"], "Arterial": ["IGFBP3","DKK2","GJA5"],
                   "Venous": ["CPE","HDAC9","ACKR1"] },
                  groupby="celltype",dendrogram=False,standard_scale="var",cmap ="RdBu_r",
                  save = "endo_marker.pdf")

In [ ]:
endo.obs['UMAP1'] = endo.obsm['X_umap'][:, 0]
endo.obs['UMAP2'] = endo.obsm['X_umap'][:, 1]
endo.obs.to_csv("./tables/endo_meta_umap.csv")
endo.write("./processed_data_py/COPD_endo.h5ad")